In [5]:
# !pip install gensim --upgrade -q

In [6]:
import gensim
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from operator import itemgetter
import tqdm
import nltk
import re
import multiprocessing

In [7]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mbaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

### Load Data

In [10]:
train_df = pd.read_csv('../../data/ag_news/train.csv')

In [11]:
for index, row in train_df.iterrows():
    if row['class'] == 1:
        train_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        train_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        train_df.at[index, 'topic'] = 'Business'
    else:
        train_df.at[index, 'topic'] = 'Sci/Tech'

In [12]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['description'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 120000/120000 [01:48<00:00, 1108.53it/s]


### Bag of Words

In [13]:
train_documents = train_df['news_tokenized'].str.split()
dictionary = gensim.corpora.Dictionary(train_documents)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=20000)

train_corpus = [dictionary.doc2bow(document) for document in train_documents]

### Find optimal epochs for NMF

In [14]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','run#', 'epoch', 'state', 'AMI','ARI','NMI'])

In [15]:
for run in range(1, 21, 1):
    print('Run #', run)
    for epoch, state in zip(range(25, 300, 15), range(2, 42, 2)):
        pred_labels = []
        gensim_nmf = gensim.models.Nmf(corpus=train_corpus, num_topics=4, id2word=dictionary, chunksize=1000, passes=epoch, eval_every=10, minimum_probability=0, 
                                   random_state=state, kappa=1)
        
        for train_doc in train_corpus:
            pred_label = max(gensim_nmf[train_doc], key=itemgetter(1))[0]
            pred_labels.append(pred_label)
        
        ami = metrics.adjusted_mutual_info_score(train_df['class'], pred_labels)
        ari = metrics.adjusted_rand_score(train_df['class'], pred_labels)
        nmi = metrics.normalized_mutual_info_score(train_df['class'], pred_labels)

        # print(run, epoch, state)
        
        performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'run#':run, 'epoch':epoch, 'state':state, 'AMI':'{:f}'.format(ami), 'ARI':'{:f}'.format(ari), 'NMI':'{:f}'.format(nmi)}, ignore_index=True)
        break

Run # 1
Run # 2


KeyboardInterrupt: 

In [ ]:
performance_metrics.to_csv('performance-bow_nmf.csv', index=False)

In [ ]:
performance_metrics[['run#','epoch','state','AMI','ARI','NMI']] = performance_metrics[['run#','epoch','state','AMI','ARI','NMI']].apply(pd.to_numeric, errors='coerce')

In [ ]:
performance_metrics.dtypes

In [ ]:
mean_performance = performance_metrics.groupby('epoch', as_index=False)[['AMI','ARI','NMI']].mean()

In [ ]:
mean_performance.to_csv('mean_performance-bow_nmf.csv', index=False)